<a href="https://colab.research.google.com/github/rcasertascuola/Python-AI-codes/blob/main/Generazione_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import random
from scipy.stats import norm, uniform, poisson

def genera_dataset(config, num_righe):
    """
    Genera un dataset basato sulla configurazione fornita.

    Args:
        config (dict): Configurazione per il dataset.
        num_righe (int): Numero di righe da generare.

    Returns:
        pd.DataFrame: Dataset generato.
    """
    # Recupera la configurazione delle colonne
    colonne = config['colonne']
    dati = {}

    # Genera i dati per ogni colonna in base al tipo e alla distribuzione specificata
    for col in colonne:
        nome_colonna = col['nome']
        tipo_colonna = col['tipo']

        if tipo_colonna == "continuo":
            # Genera dati continui secondo una distribuzione specificata
            if col['distribuzione'] == "normale":
                dati[nome_colonna] = np.clip(
                    norm.rvs(loc=col['media'], scale=col['dev_std'], size=num_righe),
                    col['min'],
                    col['max']
                )
            elif col['distribuzione'] == "uniforme":
                dati[nome_colonna] = uniform.rvs(
                    loc=col['min'],
                    scale=col['max'] - col['min'],
                    size=num_righe
                )

        elif tipo_colonna == "discreto":
            # Genera dati discreti secondo una distribuzione Poisson
            if col['distribuzione'] == "poisson":
                dati[nome_colonna] = np.clip(
                    poisson.rvs(mu=col['lambda'], size=num_righe),
                    col['min'],
                    col['max']
                )

        elif tipo_colonna == "categorico":
            # Genera dati categorici selezionando valori dalle categorie fornite
            # Normalize probabilities if they don't sum to 1
            probabilities = col['distribuzione']
            if not np.isclose(np.sum(probabilities), 1):
                probabilities = probabilities / np.sum(probabilities)

            dati[nome_colonna] = np.random.choice(
                col['categorie'], size=num_righe, p=probabilities
            )

        elif tipo_colonna == "binario":
            # Placeholder per il calcolo binario, gestito successivamente
            dati[nome_colonna] = None


    # Converte i dati generati in un DataFrame
    df = pd.DataFrame(dati)

    # Calcola la colonna binaria in base alla dipendenza specificata
    for col in colonne:
        if col['tipo'] == "binario":
            dipendenze = col['dipende_da']  # Elenco delle colonne da cui dipende
            pesi = col['pesi']  # Pesi per ciascuna colonna
            soglia = col['soglia']  # Soglia per la classificazione binaria

            # Calcola il valore pesato delle colonne dipendenti
            # valori_pesati = sum(pd.to_numeric(df[colonna], errors='coerce').mul(pesi[i]) for i, colonna in enumerate(dipendenze))

            valori_pesati = 0  # Initialize valori_pesati outside the loop

            for i, colonna in enumerate(dipendenze):
                # Get the dictionary for the current 'colonna' (if any)
                # colonna_dict = next((c['dizionario_categorie'] for c in colonne if c['nome'] == colonna), None)
                colonna_dict = next((c['dizionario_categorie'] for c in colonne if c['nome'] == colonna and 'dizionario_categorie' in c), None)
                # Apply mapping if dictionary is found, otherwise use original values
                if colonna_dict:
                    mapped_values = df[colonna].map(colonna_dict).fillna(df[colonna])
                else:
                    mapped_values = df[colonna]

                # Convert to numeric and handle errors
                numeric_values = pd.to_numeric(mapped_values, errors='coerce')

                # Multiply by weight and add to valori_pesati
                valori_pesati += numeric_values.mul(pesi[i])

            # Genera il dato binario confrontando i valori pesati con la soglia
            df[col['nome']] = valori_pesati > sum(valori_pesati) / len(valori_pesati)

            # Tiene traccia del calcolo della somma pesata
            df['Predict'] = valori_pesati

    # Introduce rumore nei dati
    for col in df.columns:
        # Introduce valori mancanti in modo casuale
        indici_mancanti = np.random.choice(df.index, size=int(0.05 * num_righe), replace=False)

        # Enforce BooleanDtype before assigning pd.NA
        if df[col].dtype == bool:
            df[col] = df[col].astype(pd.BooleanDtype())
            df.loc[indici_mancanti, col] = pd.NA  # Use pd.NA for boolean columns
        else:
            df.loc[indici_mancanti, col] = np.nan

        # Introduce errori casuali nei dati numerici
        if df[col].dtype in [np.float64, np.int64]:
            indici_errati = np.random.choice(df.index, size=int(0.02 * num_righe), replace=False)
            # Cast to the correct dtype before assigning
            #Calculate the error value first
            error_value = (df[col].mean() * random.uniform(1.5, 2.0))

            #Then cast it to the column's dtype
            casted_error_value = error_value.astype(df[col].dtype) if hasattr(error_value, 'astype') else error_value # handle cases when astype is not available

            df.loc[indici_errati, col] = casted_error_value


    return df

In [ ]:
colonne_dataset = []

colonne_dataset.append([
    {"nome": "Pioggia", "tipo": "continuo", "min": 0, "max": 200, "distribuzione": "normale", "media": 50, "dev_std": 10},
    {"nome": "Temperatura", "tipo": "continuo", "min": -20, "max": 40, "distribuzione": "uniforme"},
    {"nome": "Velocità Vento", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "uniforme"},
    {"nome": "Stagione", "tipo": "categorico", "categorie": ["Primavera", "Estate", "Autunno", "Inverno"], "dizionario_categorie": {"Primavera": 0.1, "Estate": 0.2, "Autunno": 0.3, "Inverno": 0.4}, "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Condizione Meteorologica", "tipo": "categorico", "categorie": ["Sereno", "Nuvoloso", "Piovoso"], "distribuzione": [0.4, 0.3, 0.3]},
    {"nome": "Mese", "tipo": "categorico", "categorie": ["Gennaio", "Febbraio", "Marzo", "Aprile", "Maggio", "Giugno", "Luglio", "Agosto", "Settembre", "Ottobre", "Novembre", "Dicembre"], "distribuzione": [1/12] * 12},
    {"nome": "Ore di Sole", "tipo": "continuo", "min": 0, "max": 24, "distribuzione": "uniforme"},
    {"nome": "Visibilità", "tipo": "continuo", "min": 0, "max": 10, "distribuzione": "uniforme"},
    {"nome": "Allerta Tempesta", "tipo": "binario", "dipende_da": ["Pioggia", "Temperatura", "Velocità Vento", "Stagione"], "pesi": [0.2, 0.3, -0.1, 0.1], "soglia": 5}
])

In [ ]:
stagione_colonne = [
    colonna
    for dataset in colonne_dataset
    for colonna in dataset
    if colonna['nome'] == 'Stagione'
]

stagione_colonne[0].get('dizionario_categorie', {}).get('Primavera')

0.1

In [2]:
# Lista dei titoli dei dataset
titoli_dataset = [
    "Meteo Storico", "Vendite Giornaliere", "Traffico Veicolare", "Consumi Elettrici",
    "Qualità dell'Aria", "Produzione Agricola", "Prestazioni Studenti", "Eventi Climatici",
    "Vendite Ecommerce", "Salute Pubblica", "Dati Sanitario Regionale", "Statistiche Turistiche",
    "Performance Aziendali", "Andamento Mercati Finanziari", "Comunicazioni Social Media", "Andamento Prezzi Petrolio",
    "Distribuzione Redditi", "Previsioni Meteo", "Frequenza di Visite Web", "Movimento Aeroportuale",
    "Comportamento Utenti App", "Popolazione e Demografia", "Inquinamento Acque", "Sport e Performance Atleti",
    "Analisi Logistica", "Sicurezza Stradale", "Sondaggio di Opinione", "Traffico Web per Settore"
]

#    "Monitoraggio Flora e Fauna", "Produzione Energetica Rinnovabile", "Andamento Stipendi",
#    "Statistica Criminale", "Recensioni Clienti Ecommerce", "Tendenze di Consumo", "Monitoraggio Dati Economici",
#    "Utilizzo Risorse Naturali", "Dati Elettorali", "Flusso di Passeggeri Pubblici", "Distribuzione Popolazione",
#    "Analisi Previsioni Meteorologiche", "Gestione Rifiuti", "Previsioni di Demanda", "Dati sulla Salute Mentale"


colonne_dataset = []

# Meteo Storico
colonne_dataset.append([
    {"nome": "Pioggia", "tipo": "continuo", "min": 0, "max": 200, "distribuzione": "normale", "media": 50, "dev_std": 10},
    {"nome": "Temperatura", "tipo": "continuo", "min": -20, "max": 40, "distribuzione": "uniforme"},
    {"nome": "Velocità Vento", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "uniforme"},
    {"nome": "Stagione", "tipo": "categorico", "categorie": ["Primavera", "Estate", "Autunno", "Inverno"], "dizionario_categorie": {"Primavera": 0.1, "Estate": 0.2, "Autunno": 0.3, "Inverno": 0.4}, "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Condizione Meteorologica", "tipo": "categorico", "categorie": ["Sereno", "Nuvoloso", "Piovoso"], "distribuzione": [0.4, 0.3, 0.3]},
    {"nome": "Mese", "tipo": "categorico", "categorie": ["Gennaio", "Febbraio", "Marzo", "Aprile", "Maggio", "Giugno", "Luglio", "Agosto", "Settembre", "Ottobre", "Novembre", "Dicembre"], "distribuzione": [1/12] * 12},
    {"nome": "Ore di Sole", "tipo": "continuo", "min": 0, "max": 24, "distribuzione": "uniforme"},
    {"nome": "Visibilità", "tipo": "continuo", "min": 0, "max": 10, "distribuzione": "uniforme"},
    {"nome": "Allerta Tempesta", "tipo": "binario", "dipende_da": ["Pioggia", "Temperatura", "Velocità Vento", "Stagione"], "pesi": [0.2, 0.3, -0.1, 0.1], "soglia": 5}
])

#Vendite Giornaliere
colonne_dataset.append([
    {"nome": "Fatturato", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "uniforme"},
    {"nome": "Quantità Venduta", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 500, "dev_std": 100},
    {"nome": "Sconto Medio", "tipo": "continuo", "min": 0, "max": 50, "distribuzione": "uniforme"},
    {"nome": "Numero di Ordini", "tipo": "discreto", "min": 0, "max": 1000, "distribuzione": "poisson", "lambda": 30},
    {"nome": "Resi", "tipo": "discreto", "min": 0, "max": 100, "distribuzione": "poisson", "lambda": 10},
    {"nome": "Clienti Unici", "tipo": "discreto", "min": 0, "max": 1000, "distribuzione": "poisson", "lambda": 200},
    {"nome": "Categoria Prodotto", "tipo": "categorico", "categorie": ["Elettronica", "Abbigliamento", "Cibo", "Casa", "Bellezza"], "distribuzione": [0.3, 0.2, 0.2, 0.2, 0.1]},
    {"nome": "Giorno della Settimana", "tipo": "categorico", "categorie": ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato", "Domenica"], "distribuzione": [0.15, 0.15, 0.15, 0.15, 0.15, 0.10, 0.10]},
    {"nome": "Tipo di Cliente Prevalente", "tipo": "categorico", "categorie": ["Nuovo", "Frequente"], "distribuzione": [0.6, 0.4]},
    {"nome": "Vendite Raggiunte Obiettivo", "tipo": "binario", "dipende_da": ["Fatturato", "Quantità Venduta", "Sconto Medio"], "pesi": [0.2, 0.3, -0.1], "soglia": 5}
])

# Traffico Veicolare
colonne_dataset.append([
    {"nome": "Numero di Veicoli", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 5000, "dev_std": 1000},
    {"nome": "Velocità Media", "tipo": "continuo", "min": 0, "max": 120, "distribuzione": "uniforme"},
    {"nome": "Incidenze", "tipo": "continuo", "min": 0, "max": 50, "distribuzione": "normale", "media": 5, "dev_std": 3},
    {"nome": "Tipo di Strada", "tipo": "discreto", "min": 0, "max": 10, "distribuzione": "poisson", "lambda": 8},
    {"nome": "Veicoli Sostituiti", "tipo": "discreto", "min": 0, "max": 500, "distribuzione": "poisson", "lambda": 100},
    {"nome": "Numero di Incidenti", "tipo": "discreto", "min": 0, "max": 100, "distribuzione": "poisson", "lambda": 10},
    {"nome": "Categoria Strada", "tipo": "categorico", "categorie": ["Autostrada", "Strada Urbana", "Strada Provinciale"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Ora del Giorno", "tipo": "categorico", "categorie": ["Mattina", "Pomeriggio", "Sera", "Notte"], "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Giorno della Settimana", "tipo": "categorico", "categorie": ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato", "Domenica"], "distribuzione": [0.15, 0.15, 0.15, 0.15, 0.15, 0.10, 0.10]},
    {"nome": "Allerta Traffico", "tipo": "binario", "dipende_da": ["Numero di Veicoli", "Velocità Media", "Incidenze"], "pesi": [0.2, -0.1, 0.3], "soglia": 5}
])

# Consumi Elettrici
colonne_dataset.append([
    {"nome": "Consumo Giornaliero", "tipo": "continuo", "min": 0, "max": 5000, "distribuzione": "normale", "media": 2000, "dev_std": 500},
    {"nome": "Ore di Consumo", "tipo": "continuo", "min": 0, "max": 24, "distribuzione": "uniforme"},
    {"nome": "Tipo di Apparecchio", "tipo": "discreto", "min": 0, "max": 100, "distribuzione": "poisson", "lambda": 20},
    {"nome": "Giorno della Settimana", "tipo": "discreto", "min": 0, "max": 7, "distribuzione": "poisson", "lambda": 3},
    {"nome": "Temperatura Media", "tipo": "continuo", "min": -10, "max": 40, "distribuzione": "normale", "media": 15, "dev_std": 10},
    {"nome": "Tipo di Edificio", "tipo": "categorico", "categorie": ["Residenziale", "Commerciale", "Industriale"], "distribuzione": [0.6, 0.3, 0.1]},
    {"nome": "Area Geografica", "tipo": "categorico", "categorie": ["Nord", "Centro", "Sud"], "distribuzione": [0.3, 0.3, 0.4]},
    {"nome": "Uso dell'Energia", "tipo": "categorico", "categorie": ["Riscaldamento", "Climatizzazione", "Illuminazione", "Elettrodomestici"], "distribuzione": [0.2, 0.3, 0.3, 0.2]},
    {"nome": "Fonte Energetica", "tipo": "categorico", "categorie": ["Elettricità", "Gas", "Rinnovabili"], "distribuzione": [0.7, 0.2, 0.1]},
    {"nome": "Anomalia Consumo", "tipo": "binario", "dipende_da": ["Consumo Giornaliero", "Ore di Consumo", "Temperatura Media"], "pesi": [0.2, 0.1, 0.3], "soglia": 10}
])

# Qualità dell'Aria
colonne_dataset.append([
    {"nome": "PM10", "tipo": "continuo", "min": 0, "max": 500, "distribuzione": "normale", "media": 50, "dev_std": 15},
    {"nome": "PM2.5", "tipo": "continuo", "min": 0, "max": 200, "distribuzione": "normale", "media": 25, "dev_std": 10},
    {"nome": "CO2", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 300, "dev_std": 100},
    {"nome": "Ore di Inquinamento", "tipo": "discreto", "min": 0, "max": 24, "distribuzione": "poisson", "lambda": 5},
    {"nome": "Indice di Qualità dell'Aria", "tipo": "discreto", "min": 0, "max": 500, "distribuzione": "poisson", "lambda": 100},
    {"nome": "Temperatura", "tipo": "continuo", "min": -20, "max": 40, "distribuzione": "uniforme"},
    {"nome": "Umidità Relativa", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "uniforme"},
    {"nome": "Tipo di Aria", "tipo": "categorico", "categorie": ["Città", "Rurale", "Industriale"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Stagione", "tipo": "categorico", "categorie": ["Primavera", "Estate", "Autunno", "Inverno"], "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Allerta Smog", "tipo": "binario", "dipende_da": ["PM10", "PM2.5", "CO2"], "pesi": [0.3, 0.4, 0.2], "soglia": 50}
])

# Produzione Agricola
colonne_dataset.append([
    {"nome": "Produzione (kg)", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 5000, "dev_std": 1500},
    {"nome": "Prezzo (€/kg)", "tipo": "continuo", "min": 0, "max": 5, "distribuzione": "uniforme"},
    {"nome": "Area Coltivata (ha)", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 100, "dev_std": 25},
    {"nome": "Tipo di Coltura", "tipo": "discreto", "min": 0, "max": 10, "distribuzione": "poisson", "lambda": 3},
    {"nome": "Numero di Lavoratori", "tipo": "discreto", "min": 0, "max": 100, "distribuzione": "poisson", "lambda": 30},
    {"nome": "Stagione di Raccolta", "tipo": "discreto", "min": 0, "max": 12, "distribuzione": "poisson", "lambda": 6},
    {"nome": "Tipo di Terreno", "tipo": "categorico", "categorie": ["Agricolo", "Boschivo", "Montano"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Regione", "tipo": "categorico", "categorie": ["Nord", "Centro", "Sud"], "distribuzione": [0.3, 0.4, 0.3]},
    {"nome": "Metodo di Coltivazione", "tipo": "categorico", "categorie": ["Biologico", "Tradizionale"], "distribuzione": [0.7, 0.3]},
    {"nome": "Successo Produzione Stagionale", "tipo": "binario", "dipende_da": ["Produzione (kg)", "Area Coltivata (ha)", "Tipo di Coltura"], "pesi": [0.4, 0.3, 0.3], "soglia": 5000}
])


# Prestazioni Studenti
colonne_dataset.append([
    {"nome": "Punteggio Esame", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "normale", "media": 75, "dev_std": 10},
    {"nome": "Ore di Studio", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "uniforme"},
    {"nome": "Assenze", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "normale", "media": 5, "dev_std": 3},
    {"nome": "Anno di Studio", "tipo": "discreto", "min": 1, "max": 5, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Media Voti", "tipo": "discreto", "min": 0, "max": 30, "distribuzione": "poisson", "lambda": 20},
    {"nome": "Ore di Lezione", "tipo": "discreto", "min": 0, "max": 30, "distribuzione": "poisson", "lambda": 15},
    {"nome": "Facoltà", "tipo": "categorico", "categorie": ["Ingegneria", "Medicina", "Economia", "Giurisprudenza", "Arte"], "distribuzione": [0.25, 0.2, 0.2, 0.15, 0.2]},
    {"nome": "Età", "tipo": "categorico", "categorie": ["18-22", "23-27", "28-32"], "distribuzione": [0.4, 0.4, 0.2]},
    {"nome": "Tipo di Corso", "tipo": "categorico", "categorie": ["Teorico", "Pratico"], "distribuzione": [0.6, 0.4]},
    {"nome": "Successo Accademico", "tipo": "binario", "dipende_da": ["Punteggio Esame", "Ore di Studio", "Assenze"], "pesi": [0.3, 0.4, -0.1], "soglia": 10}
])

# Eventi Climatici
colonne_dataset.append([
    {"nome": "Temperatura Media", "tipo": "continuo", "min": -50, "max": 50, "distribuzione": "normale", "media": 25, "dev_std": 15},
    {"nome": "Precipitazioni", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 200, "dev_std": 100},
    {"nome": "Velocità del Vento", "tipo": "continuo", "min": 0, "max": 200, "distribuzione": "uniforme"},
    {"nome": "Tipo di Evento", "tipo": "discreto", "min": 0, "max": 10, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Durata (Ore)", "tipo": "discreto", "min": 0, "max": 72, "distribuzione": "poisson", "lambda": 6},
    {"nome": "Intensità", "tipo": "discreto", "min": 0, "max": 5, "distribuzione": "poisson", "lambda": 3},
    {"nome": "Giorno della Settimana", "tipo": "categorico", "categorie": ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato", "Domenica"], "distribuzione": [0.15, 0.15, 0.15, 0.15, 0.15, 0.10, 0.10]},
    {"nome": "Stagione", "tipo": "categorico", "categorie": ["Primavera", "Estate", "Autunno", "Inverno"], "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Evento Estremo", "tipo": "binario", "dipende_da": ["Temperatura Media", "Precipitazioni", "Velocità del Vento"], "pesi": [0.3, 0.2, 0.1], "soglia": 50}
])

# Vendite Ecommerce
colonne_dataset.append([
    {"nome": "Vendite Giornalieri (unità)", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 250, "dev_std": 50},
    {"nome": "Prezzo Medio (€/unità)", "tipo": "continuo", "min": 5, "max": 500, "distribuzione": "uniforme"},
    {"nome": "Visite Sito Web", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 2000, "dev_std": 500},
    {"nome": "Categoria Prodotto", "tipo": "discreto", "min": 0, "max": 5, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Numero di Recensioni", "tipo": "discreto", "min": 0, "max": 100, "distribuzione": "poisson", "lambda": 30},
    {"nome": "Tempi di Consegna (giorni)", "tipo": "discreto", "min": 1, "max": 10, "distribuzione": "poisson", "lambda": 3},
    {"nome": "Tipo di Prodotto", "tipo": "categorico", "categorie": ["Elettronica", "Abbigliamento", "Casa", "Alimentari"], "distribuzione": [0.4, 0.3, 0.2, 0.1]},
    {"nome": "Metodo di Pagamento", "tipo": "categorico", "categorie": ["Carta di Credito", "PayPal", "Bonifico", "Contrassegno"], "distribuzione": [0.5, 0.3, 0.1, 0.1]},
    {"nome": "Sconto Applicato", "tipo": "categorico", "categorie": ["No", "Sì"], "distribuzione": [0.7, 0.3]},
    {"nome": "Transazione Rimborsata", "tipo": "binario", "dipende_da": ["Vendite Giornalieri (unità)", "Prezzo Medio (€/unità)", "Numero di Recensioni"], "pesi": [0.5, 0.3, 0.2], "soglia": 1000}
])

# Salute Pubblica
colonne_dataset.append([
    {"nome": "Tasso di Mortalità", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "normale", "media": 5, "dev_std": 1.5},
    {"nome": "Numero di Visite Ospedaliere", "tipo": "continuo", "min": 0, "max": 5000, "distribuzione": "uniforme"},
    {"nome": "Incidenza di Malattie Cardiovascolari", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 200, "dev_std": 50},
    {"nome": "Fascia di Età", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Stato di Salute", "tipo": "discreto", "min": 0, "max": 5, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Numero di Vaccinazioni", "tipo": "discreto", "min": 0, "max": 1000, "distribuzione": "poisson", "lambda": 100},
    {"nome": "Tipo di Malattia", "tipo": "categorico", "categorie": ["Cardiovascolare", "Respiratoria", "Oncologica"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Area Geografica", "tipo": "categorico", "categorie": ["Nord", "Centro", "Sud"], "distribuzione": [0.3, 0.4, 0.3]},
    {"nome": "Stato Economico", "tipo": "categorico", "categorie": ["Basso", "Medio", "Alto"], "distribuzione": [0.4, 0.4, 0.2]},
    {"nome": "Emergenza Sanitaria", "tipo": "binario", "dipende_da": ["Numero di Visite Ospedaliere", "Incidenza di Malattie Cardiovascolari", "Fascia di Età"], "pesi": [0.4, 0.3, 0.3], "soglia": 500}
])

# Traffico Veicolare
colonne_dataset.append([
    {"nome": "Numero Veicoli", "tipo": "continuo", "min": 0, "max": 5000, "distribuzione": "normale", "media": 1500, "dev_std": 500},
    {"nome": "Tempo di Viaggio (minuti)", "tipo": "continuo", "min": 0, "max": 120, "distribuzione": "uniforme"},
    {"nome": "Velocità Media (km/h)", "tipo": "continuo", "min": 0, "max": 120, "distribuzione": "normale", "media": 60, "dev_std": 20},
    {"nome": "Tipo di Strada", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Incidente Stradale", "tipo": "discreto", "min": 0, "max": 50, "distribuzione": "poisson", "lambda": 5},
    {"nome": "Tempo di Attesa", "tipo": "discreto", "min": 0, "max": 60, "distribuzione": "poisson", "lambda": 10},
    {"nome": "Tipo di Veicolo", "tipo": "categorico", "categorie": ["Auto", "Furgone", "Camion"], "distribuzione": [0.6, 0.3, 0.1]},
    {"nome": "Ora del Giorno", "tipo": "categorico", "categorie": ["Mattina", "Pomeriggio", "Sera", "Notte"], "distribuzione": [0.3, 0.4, 0.2, 0.1]},
    {"nome": "Tipo di Viaggio", "tipo": "categorico", "categorie": ["Pendolare", "Lavoro", "Turismo"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Congestionamento", "tipo": "binario", "dipende_da": ["Numero Veicoli", "Tempo di Viaggio (minuti)", "Velocità Media (km/h)"], "pesi": [0.4, 0.3, 0.3], "soglia": 1000}
])

# Consumi Elettrici
colonne_dataset.append([
    {"nome": "Consumo Giornaliero (kWh)", "tipo": "continuo", "min": 0, "max": 5000, "distribuzione": "normale", "media": 2000, "dev_std": 500},
    {"nome": "Numero di Abitazioni", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "uniforme"},
    {"nome": "Temperatura Media Giornaliera", "tipo": "continuo", "min": -10, "max": 40, "distribuzione": "normale", "media": 15, "dev_std": 5},
    {"nome": "Tipo di Settore", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Fascia Oraria", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Consumo per Settore", "tipo": "discreto", "min": 0, "max": 5000, "distribuzione": "poisson", "lambda": 1000},
    {"nome": "Giorno della Settimana", "tipo": "categorico", "categorie": ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato", "Domenica"], "distribuzione": [0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14]},
    {"nome": "Fonte di Energia", "tipo": "categorico", "categorie": ["Eolica", "Solare", "Fossile", "Idroelettrica"], "distribuzione": [0.3, 0.2, 0.4, 0.1]},
    {"nome": "Tipo di Contratto", "tipo": "categorico", "categorie": ["Residenziale", "Commerciale", "Industriale"], "distribuzione": [0.6, 0.3, 0.1]},
    {"nome": "Efficienza Energetica", "tipo": "binario", "dipende_da": ["Consumo Giornaliero (kWh)", "Temperatura Media Giornaliera", "Fascia Oraria"], "pesi": [0.5, 0.3, 0.2], "soglia": 1500}
])

# Vendite Giornaliere
colonne_dataset.append([
    {"nome": "Numero di Vendite", "tipo": "continuo", "min": 0, "max": 5000, "distribuzione": "normale", "media": 1000, "dev_std": 300},
    {"nome": "Totale Vendite (€)", "tipo": "continuo", "min": 0, "max": 100000, "distribuzione": "uniforme"},
    {"nome": "Sconto Applicato (%)", "tipo": "continuo", "min": 0, "max": 50, "distribuzione": "normale", "media": 10, "dev_std": 5},
    {"nome": "Giorno della Settimana", "tipo": "discreto", "min": 0, "max": 6, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Tipo di Prodotto", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Promozione Attiva", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Categoria Prodotto", "tipo": "categorico", "categorie": ["Elettronica", "Abbigliamento", "Cibo", "Libri"], "distribuzione": [0.3, 0.3, 0.2, 0.2]},
    {"nome": "Canale di Vendita", "tipo": "categorico", "categorie": ["Online", "In Store", "Misto"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Fascia Oraria", "tipo": "categorico", "categorie": ["Mattina", "Pomeriggio", "Sera"], "distribuzione": [0.4, 0.4, 0.2]},
    {"nome": "Superficie di Vendita (mq)", "tipo": "binario", "dipende_da": ["Numero di Vendite", "Sconto Applicato (%)", "Tipo di Prodotto"], "pesi": [0.4, 0.3, 0.3], "soglia": 500}
])

# Statistiche Turistiche
colonne_dataset.append([
    {"nome": "Numero di Visitatori", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 50000, "dev_std": 10000},
    {"nome": "Soggiorno Medio (giorni)", "tipo": "continuo", "min": 0, "max": 30, "distribuzione": "uniforme"},
    {"nome": "Entrate da Turismo (Euro)", "tipo": "continuo", "min": 0, "max": 10000000, "distribuzione": "normale", "media": 500000, "dev_std": 100000},
    {"nome": "Tipo di Attrazione", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Mese", "tipo": "discreto", "min": 0, "max": 11, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Anno", "tipo": "discreto", "min": 2020, "max": 2024, "distribuzione": "uniforme"},
    {"nome": "Provenienza dei Visitatori", "tipo": "categorico", "categorie": ["Internazionale", "Locale", "Regionale"], "distribuzione": [0.3, 0.4, 0.3]},
    {"nome": "Metodo di Trasporto", "tipo": "categorico", "categorie": ["Auto", "Treno", "Aereo", "Nave"], "distribuzione": [0.4, 0.2, 0.3, 0.1]},
    {"nome": "Alloggio", "tipo": "categorico", "categorie": ["Hotel", "B&B", "Appartamento", "Ostelli"], "distribuzione": [0.5, 0.2, 0.2, 0.1]},
    {"nome": "Soddisfazione Cliente", "tipo": "binario", "dipende_da": ["Numero di Visitatori", "Entrate da Turismo (Euro)", "Tipo di Attrazione"], "pesi": [0.5, 0.3, 0.2], "soglia": 100000}
])

# Performance Aziendali
colonne_dataset.append([
    {"nome": "Ricavi Totali (Euro)", "tipo": "continuo", "min": 0, "max": 100000000, "distribuzione": "normale", "media": 5000000, "dev_std": 1000000},
    {"nome": "Profitti (Euro)", "tipo": "continuo", "min": -10000000, "max": 50000000, "distribuzione": "normale", "media": 2000000, "dev_std": 500000},
    {"nome": "Dipendenti", "tipo": "discreto", "min": 0, "max": 10000, "distribuzione": "poisson", "lambda": 300},
    {"nome": "Anno", "tipo": "discreto", "min": 2020, "max": 2024, "distribuzione": "uniforme"},
    {"nome": "Settore", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Rendimento del Capitale", "tipo": "continuo", "min": -10, "max": 30, "distribuzione": "uniforme"},
    {"nome": "Soddisfazione dei Clienti", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "normale", "media": 80, "dev_std": 5},
    {"nome": "Quota di Mercato", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "uniforme"},
    {"nome": "Crescita Fatturato (%)", "tipo": "continuo", "min": -20, "max": 100, "distribuzione": "normale", "media": 10, "dev_std": 5},
    {"nome": "Obiettivi Raggiunti", "tipo": "binario", "dipende_da": ["Ricavi Totali (Euro)", "Profitti (Euro)", "Crescita Fatturato (%)"], "pesi": [0.5, 0.3, 0.2], "soglia": 5000000}
])

# Andamento Mercati Finanziari
colonne_dataset.append([
    {"nome": "Indice Azionario", "tipo": "continuo", "min": 1000, "max": 50000, "distribuzione": "normale", "media": 15000, "dev_std": 3000},
    {"nome": "Volume di Scambi", "tipo": "continuo", "min": 0, "max": 10000000, "distribuzione": "normale", "media": 1000000, "dev_std": 500000},
    {"nome": "Variazione Percentuale", "tipo": "continuo", "min": -10, "max": 10, "distribuzione": "uniforme"},
    {"nome": "Settore", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Data", "tipo": "discreto", "min": 2020, "max": 2024, "distribuzione": "uniforme"},
    {"nome": "Banca Centrale", "tipo": "categorico", "categorie": ["FED", "ECB", "BoJ"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Tasso di Interesse", "tipo": "continuo", "min": -5, "max": 10, "distribuzione": "normale", "media": 2, "dev_std": 1},
    {"nome": "Inflazione (%)", "tipo": "continuo", "min": 0, "max": 15, "distribuzione": "normale", "media": 2, "dev_std": 1},
    {"nome": "Divisa", "tipo": "categorico", "categorie": ["USD", "EUR", "JPY"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Performance del Mercato", "tipo": "binario", "dipende_da": ["Indice Azionario", "Volume di Scambi", "Variazione Percentuale"], "pesi": [0.4, 0.4, 0.2], "soglia": 10}
])

# Comunicazioni Social Media
colonne_dataset.append([
    {"nome": "Numero di Post", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 500, "dev_std": 100},
    {"nome": "Interazioni", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 50000, "dev_std": 10000},
    {"nome": "Frequenza di Pubblicazione", "tipo": "continuo", "min": 0, "max": 24, "distribuzione": "uniforme"},
    {"nome": "Tipo di Contenuto", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Piattaforma", "tipo": "categorico", "categorie": ["Facebook", "Twitter", "Instagram", "LinkedIn"], "distribuzione": [0.4, 0.3, 0.2, 0.1]},
    {"nome": "Hashtag Utilizzati", "tipo": "discreto", "min": 0, "max": 10, "distribuzione": "poisson", "lambda": 3},
    {"nome": "Target di Pubblico", "tipo": "categorico", "categorie": ["Giovani", "Adulti", "Anziani"], "distribuzione": [0.5, 0.4, 0.1]},
    {"nome": "Durata Media Post (min)", "tipo": "continuo", "min": 0, "max": 15, "distribuzione": "normale", "media": 5, "dev_std": 1},
    {"nome": "Interazione Media per Post", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 500, "dev_std": 100},
    {"nome": "Impressioni Totali", "tipo": "continuo", "min": 0, "max": 10000000, "distribuzione": "normale", "media": 1000000, "dev_std": 200000}
])

# Andamento Prezzi Petrolio
colonne_dataset.append([
    {"nome": "Prezzo Petrolio (USD)", "tipo": "continuo", "min": 20, "max": 150, "distribuzione": "normale", "media": 60, "dev_std": 10},
    {"nome": "Volume di Scambi (barili)", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 50000, "dev_std": 10000},
    {"nome": "Produzione Petrolio (barili)", "tipo": "continuo", "min": 100000, "max": 10000000, "distribuzione": "normale", "media": 500000, "dev_std": 200000},
    {"nome": "Tipo di Raffinazione", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Zona di Produzione", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Mese", "tipo": "discreto", "min": 0, "max": 11, "distribuzione": "uniforme"},
    {"nome": "Conflitti Geopolitici", "tipo": "binario", "dipende_da": ["Produzione Petrolio (barili)", "Volume di Scambi (barili)", "Prezzo Petrolio (USD)"], "pesi": [0.5, 0.3, 0.2], "soglia": 50},
    {"nome": "Domanda di Petrolio", "tipo": "continuo", "min": 0, "max": 10000000, "distribuzione": "normale", "media": 3000000, "dev_std": 500000},
    {"nome": "Quota di Mercato OPEC", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "uniforme"},
    {"nome": "Tendenza dei Prezzi", "tipo": "binario", "dipende_da": ["Prezzo Petrolio (USD)", "Domanda di Petrolio"], "pesi": [0.4, 0.6], "soglia": 70}
])

# Distribuzione Redditi
colonne_dataset.append([
    {"nome": "Reddito Medio (Euro)", "tipo": "continuo", "min": 0, "max": 100000, "distribuzione": "normale", "media": 25000, "dev_std": 5000},
    {"nome": "Reddito Annuo (Euro)", "tipo": "continuo", "min": 1000, "max": 1000000, "distribuzione": "normale", "media": 30000, "dev_std": 15000},
    {"nome": "Numero di Persone per Fascia di Reddito", "tipo": "discreto", "min": 0, "max": 100000, "distribuzione": "poisson", "lambda": 5000},
    {"nome": "Anno", "tipo": "discreto", "min": 2020, "max": 2024, "distribuzione": "uniforme"},
    {"nome": "Fascia di Reddito", "tipo": "categorico", "categorie": ["Basso", "Medio", "Alto"], "distribuzione": [0.4, 0.5, 0.1]},
    {"nome": "Livello di Educazione", "tipo": "categorico", "categorie": ["Diploma", "Laurea", "Post Laurea"], "distribuzione": [0.3, 0.5, 0.2]},
    {"nome": "Settore Lavorativo", "tipo": "categorico", "categorie": ["Agricoltura", "Industria", "Servizi", "Tecnologia"], "distribuzione": [0.2, 0.3, 0.4, 0.1]},
    {"nome": "Fonte di Reddito Primaria", "tipo": "categorico", "categorie": ["Salario", "Investimenti", "Pensioni"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Tassazione Media (%)", "tipo": "continuo", "min": 0, "max": 50, "distribuzione": "normale", "media": 20, "dev_std": 5},
    {"nome": "Distribuzione Territoriale", "tipo": "binario", "dipende_da": ["Fascia di Reddito", "Settore Lavorativo", "Reddito Annuo (Euro)"], "pesi": [0.4, 0.4, 0.2], "soglia": 30000}
])

# Frequenza di Visite Web
colonne_dataset.append([
    {"nome": "Visite Giornalieri", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 50000, "dev_std": 10000},
    {"nome": "Durata Media Visite (min)", "tipo": "continuo", "min": 0, "max": 60, "distribuzione": "uniforme"},
    {"nome": "Fonte di Traffico", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 2},
    {"nome": "Tipo di Contenuto Visto", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Tipo di Dispositivo", "tipo": "categorico", "categorie": ["Desktop", "Mobile", "Tablet"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Fascia Oraria di Visita", "tipo": "categorico", "categorie": ["Mattina", "Pomeriggio", "Sera", "Notte"], "distribuzione": [0.2, 0.3, 0.3, 0.2]},
    {"nome": "Tipo di Visitante", "tipo": "categorico", "categorie": ["Nuovo", "Ritorno"], "distribuzione": [0.7, 0.3]},
    {"nome": "Pagine Visitate", "tipo": "discreto", "min": 1, "max": 20, "distribuzione": "poisson", "lambda": 5},
    {"nome": "Frequenza di Ritorno (giorni)", "tipo": "continuo", "min": 0, "max": 30, "distribuzione": "uniforme"},
    {"nome": "Tasso di Conversione", "tipo": "continuo", "min": 0, "max": 1, "distribuzione": "normale", "media": 0.05, "dev_std": 0.02}
])

# Movimento Aeroportuale
colonne_dataset.append([
    {"nome": "Numero di Voli", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 500, "dev_std": 100},
    {"nome": "Numero di Passeggeri", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 20000, "dev_std": 5000},
    {"nome": "Tempo Medio di Attesa (min)", "tipo": "continuo", "min": 0, "max": 180, "distribuzione": "uniforme"},
    {"nome": "Tipo di Volo", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Fascia Oraria di Partenza", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Stato del Volo", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["In Orario", "Ritardato", "Annullato", "In Anticipo"]},
    {"nome": "Aeroporto di Origine", "tipo": "categorico", "categorie": ["Aeroporto A", "Aeroporto B", "Aeroporto C", "Aeroporto D"], "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Aeroporto di Destinazione", "tipo": "categorico", "categorie": ["Aeroporto A", "Aeroporto B", "Aeroporto C", "Aeroporto D"], "distribuzione": [0.25, 0.25, 0.25, 0.25]},
    {"nome": "Voli Internazionali", "tipo": "binario", "dipende_da": ["Numero di Voli", "Tipo di Volo"], "pesi": [0.7, 0.3], "soglia": 60},
    {"nome": "Ritardi Media (min)", "tipo": "continuo", "min": 0, "max": 300, "distribuzione": "normale", "media": 30, "dev_std": 10}
])

# Comportamento Utenti App
colonne_dataset.append([
    {"nome": "Tempo Medio di Utilizzo (min)", "tipo": "continuo", "min": 0, "max": 240, "distribuzione": "uniforme"},
    {"nome": "Numero di Sessioni", "tipo": "continuo", "min": 0, "max": 1000, "distribuzione": "normale", "media": 100, "dev_std": 20},
    {"nome": "Durata Media Sessione (min)", "tipo": "continuo", "min": 0, "max": 60, "distribuzione": "uniforme"},
    {"nome": "Tipo di App", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Frequenza di Ritorno", "tipo": "discreto", "min": 0, "max": 7, "distribuzione": "poisson", "lambda": 3},
    {"nome": "Sistema Operativo", "tipo": "categorico", "categorie": ["iOS", "Android", "Windows"], "distribuzione": [0.4, 0.5, 0.1]},
    {"nome": "Azioni In-App", "tipo": "categorico", "categorie": ["Acquisti", "Visualizzazione", "Interazione", "Nessuna"], "distribuzione": [0.2, 0.3, 0.4, 0.1]},
    {"nome": "Caratteristiche Utente", "tipo": "categorico", "categorie": ["Nuovo", "Ritornato"], "distribuzione": [0.6, 0.4]},
    {"nome": "Versione App", "tipo": "discreto", "min": 1, "max": 10, "distribuzione": "poisson", "lambda": 4},
    {"nome": "Livello di Soddisfazione", "tipo": "continuo", "min": 1, "max": 10, "distribuzione": "normale", "media": 7, "dev_std": 1.5}
])
# Popolazione e Demografia
colonne_dataset.append([
    {"nome": "Popolazione Totale", "tipo": "continuo", "min": 10000, "max": 1000000000, "distribuzione": "normale", "media": 50000000, "dev_std": 20000000},
    {"nome": "Fascia di Età", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "uniforme", "categorie": ["0-14", "15-24", "25-64", "65+"]},
    {"nome": "Distribuzione Genere", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "uniforme", "categorie": ["Maschile", "Femminile"]},
    {"nome": "Tasso di Crescita Annuo (%)", "tipo": "continuo", "min": -10, "max": 10, "distribuzione": "uniforme"},
    {"nome": "Numero di Nascite", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 50000, "dev_std": 10000},
    {"nome": "Numero di Morti", "tipo": "continuo", "min": 0, "max": 1000000, "distribuzione": "normale", "media": 40000, "dev_std": 8000},
    {"nome": "Percentuale di Immigrati", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "uniforme"},
    {"nome": "Livello di Istruzione", "tipo": "categorico", "categorie": ["Nessuna", "Scuola", "Laurea", "Post Laurea"], "distribuzione": [0.1, 0.3, 0.4, 0.2]},
    {"nome": "Occupazione", "tipo": "categorico", "categorie": ["Occupato", "Disoccupato", "Inattivo"], "distribuzione": [0.6, 0.2, 0.2]},
    {"nome": "Area Geografica", "tipo": "categorico", "categorie": ["Urbano", "Rurale"], "distribuzione": [0.7, 0.3]}
])

# Sport e Performance Atleti
colonne_dataset.append([
    {"nome": "Tempo di Allenamento (ore)", "tipo": "continuo", "min": 0, "max": 40, "distribuzione": "uniforme"},
    {"nome": "Numero di Competizioni", "tipo": "continuo", "min": 0, "max": 200, "distribuzione": "normale", "media": 50, "dev_std": 20},
    {"nome": "Performance (Punteggio)", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "normale", "media": 80, "dev_std": 10},
    {"nome": "Sport Praticato", "tipo": "discreto", "min": 0, "max": 5, "distribuzione": "uniforme", "categorie": ["Corsa", "Nuoto", "Pallacanestro", "Calcio", "Tennis"]},
    {"nome": "Tipo di Allenamento", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "poisson", "lambda": 1},
    {"nome": "Alimentazione", "tipo": "categorico", "categorie": ["Vegetariana", "Carne", "Vegana", "Mista"], "distribuzione": [0.2, 0.4, 0.2, 0.2]},
    {"nome": "Salute Generale", "tipo": "continuo", "min": 0, "max": 100, "distribuzione": "normale", "media": 80, "dev_std": 10},
    {"nome": "Sesso Atleta", "tipo": "categorico", "categorie": ["Maschile", "Femminile"], "distribuzione": [0.5, 0.5]},
    {"nome": "Fascia di Età Atleta", "tipo": "categorico", "categorie": ["15-24", "25-34", "35-44", "45+"], "distribuzione": [0.3, 0.4, 0.2, 0.1]},
    {"nome": "Infortunio Durante l'Anno", "tipo": "binario", "dipende_da": ["Tempo di Allenamento (ore)", "Performance (Punteggio)"], "pesi": [0.6, 0.4], "soglia": 50}
])

# Analisi Logistica
colonne_dataset.append([
    {"nome": "Tempo di Consegna (giorni)", "tipo": "continuo", "min": 1, "max": 30, "distribuzione": "normale", "media": 7, "dev_std": 2},
    {"nome": "Quantità di Merce Spedita", "tipo": "continuo", "min": 0, "max": 100000, "distribuzione": "normale", "media": 5000, "dev_std": 1000},
    {"nome": "Numero di Resi", "tipo": "continuo", "min": 0, "max": 5000, "distribuzione": "poisson", "lambda": 50},
    {"nome": "Tipo di Prodotto", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Elettronica", "Abbigliamento", "Alimentari", "Arredamento"]},
    {"nome": "Canale di Vendita", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "poisson", "lambda": 1, "categorie": ["Online", "Fisico"]},
    {"nome": "Metodo di Spedizione", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Standard", "Espresso", "Ritiro in Negozio", "Paccocelere"]},
    {"nome": "Tipo di Pagamento", "tipo": "categorico", "categorie": ["Carta di Credito", "Bonifico", "Contrassegno"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Indirizzo di Consegna", "tipo": "categorico", "categorie": ["Urbano", "Rurale"], "distribuzione": [0.7, 0.3]},
    {"nome": "Problemi durante la Spedizione", "tipo": "binario", "dipende_da": ["Tempo di Consegna (giorni)", "Quantità di Merce Spedita"], "pesi": [0.4, 0.6], "soglia": 20},
    {"nome": "Soddisfazione Cliente", "tipo": "continuo", "min": 0, "max": 10, "distribuzione": "normale", "media": 8, "dev_std": 1.5}
])

# Sicurezza Stradale
colonne_dataset.append([
    {"nome": "Numero di Incidenti", "tipo": "continuo", "min": 0, "max": 10000, "distribuzione": "normale", "media": 500, "dev_std": 100},
    {"nome": "Severità Incidente", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Lieve", "Moderato", "Grave"]},
    {"nome": "Tipo di Veicolo Coinvolto", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Auto", "Camion", "Moto"]},
    {"nome": "Condizioni Meteo", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Sole", "Pioggia", "Neve"]},
    {"nome": "Ora del Giorno", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Mattina", "Pomeriggio", "Sera"]},
    {"nome": "Velocità Media", "tipo": "continuo", "min": 0, "max": 180, "distribuzione": "normale", "media": 70, "dev_std": 20},
    {"nome": "Tipologia di Strada", "tipo": "categorico", "categorie": ["Autostrada", "Strada Statale", "Strada Comunale"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Sesso del Conducente", "tipo": "categorico", "categorie": ["Maschile", "Femminile"], "distribuzione": [0.7, 0.3]},
    {"nome": "Uso di Alcol", "tipo": "binario", "dipende_da": ["Velocità Media", "Numero di Incidenti"], "pesi": [0.8, 0.2], "soglia": 50},
    {"nome": "Età del Conducente", "tipo": "continuo", "min": 18, "max": 80, "distribuzione": "normale", "media": 35, "dev_std": 10}
])

# Sondaggio di Opinione
colonne_dataset.append([
    {"nome": "Numero di Risposte", "tipo": "continuo", "min": 0, "max": 100000, "distribuzione": "normale", "media": 5000, "dev_std": 1000},
    {"nome": "Età Media Rispondenti", "tipo": "continuo", "min": 18, "max": 80, "distribuzione": "normale", "media": 40, "dev_std": 12},
    {"nome": "Soddisfazione Generale", "tipo": "continuo", "min": 1, "max": 10, "distribuzione": "normale", "media": 7, "dev_std": 2},
    {"nome": "Settore di Interesse", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "uniforme", "categorie": ["Salute", "Politica", "Educazione", "Ambiente"]},
    {"nome": "Posizione Geografica", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Nord", "Centro", "Sud", "Isole"]},
    {"nome": "Frequenza Partecipazione", "tipo": "discreto", "min": 0, "max": 3, "distribuzione": "uniforme", "categorie": ["Una volta", "Più volte", "Mai"]},
    {"nome": "Metodo di Compilazione", "tipo": "categorico", "categorie": ["Online", "Cartaceo"], "distribuzione": [0.6, 0.4]},
    {"nome": "Affiliazione Politica", "tipo": "categorico", "categorie": ["Liberale", "Conservatore", "Socialista", "Indipendente"], "distribuzione": [0.3, 0.3, 0.2, 0.2]},
    {"nome": "Soddisfazione con il Governo", "tipo": "binario", "dipende_da": ["Età Media Rispondenti", "Soddisfazione Generale"], "pesi": [0.7, 0.3], "soglia": 50},
    {"nome": "Interesse per il Futuro", "tipo": "continuo", "min": 1, "max": 10, "distribuzione": "normale", "media": 6, "dev_std": 1.5}
])

# Traffico Web per Settore
colonne_dataset.append([
    {"nome": "Visite Giornaliere", "tipo": "continuo", "min": 100, "max": 100000, "distribuzione": "normale", "media": 5000, "dev_std": 1000},
    {"nome": "Numero di Pagine Visitate", "tipo": "continuo", "min": 1, "max": 50, "distribuzione": "normale", "media": 15, "dev_std": 5},
    {"nome": "Tempo Medio di Permanenza (min)", "tipo": "continuo", "min": 1, "max": 30, "distribuzione": "normale", "media": 10, "dev_std": 3},
    {"nome": "Settore Aziendale", "tipo": "discreto", "min": 0, "max": 4, "distribuzione": "uniforme", "categorie": ["Tecnologia", "Moda", "Alimentari", "Sanità"]},
    {"nome": "Tipo di Contenuto", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "poisson", "lambda": 1, "categorie": ["Blog", "E-commerce", "News"]},
    {"nome": "Dispositivo di Accesso", "tipo": "discreto", "min": 0, "max": 2, "distribuzione": "uniforme", "categorie": ["Desktop", "Mobile"]},
    {"nome": "Sorgente del Traffico", "tipo": "categorico", "categorie": ["Organico", "Paga", "Social Media"], "distribuzione": [0.4, 0.3, 0.3]},
    {"nome": "Provenienza Geografica", "tipo": "categorico", "categorie": ["Italia", "Europa", "Extra UE"], "distribuzione": [0.5, 0.3, 0.2]},
    {"nome": "Tasso di Conversione", "tipo": "binario", "dipende_da": ["Visite Giornaliere", "Numero di Pagine Visitate"], "pesi": [0.7, 0.3], "soglia": 50},
    {"nome": "Soddisfazione del Visitatore", "tipo": "continuo", "min": 1, "max": 10, "distribuzione": "normale", "media": 8, "dev_std": 1}
])



# Stampa per visualizzare le liste
print("Titoli dei dataset:")
print(titoli_dataset)

print("\nColonne per ciascun dataset:")
for i, colonne in enumerate(colonne_dataset, 1):
    print(f"\nDataset {i} ({titoli_dataset[i-1]})")
#     for col in colonne:
#         print(f"  - {col['nome']} ({col['tipo']})")


Titoli dei dataset:
['Meteo Storico', 'Vendite Giornaliere', 'Traffico Veicolare', 'Consumi Elettrici', "Qualità dell'Aria", 'Produzione Agricola', 'Prestazioni Studenti', 'Eventi Climatici', 'Vendite Ecommerce', 'Salute Pubblica', 'Dati Sanitario Regionale', 'Statistiche Turistiche', 'Performance Aziendali', 'Andamento Mercati Finanziari', 'Comunicazioni Social Media', 'Andamento Prezzi Petrolio', 'Distribuzione Redditi', 'Previsioni Meteo', 'Frequenza di Visite Web', 'Movimento Aeroportuale', 'Comportamento Utenti App', 'Popolazione e Demografia', 'Inquinamento Acque', 'Sport e Performance Atleti', 'Analisi Logistica', 'Sicurezza Stradale', 'Sondaggio di Opinione', 'Traffico Web per Settore']

Colonne per ciascun dataset:

Dataset 1 (Meteo Storico)

Dataset 2 (Vendite Giornaliere)

Dataset 3 (Traffico Veicolare)

Dataset 4 (Consumi Elettrici)

Dataset 5 (Qualità dell'Aria)

Dataset 6 (Produzione Agricola)

Dataset 7 (Prestazioni Studenti)

Dataset 8 (Eventi Climatici)

Dataset 9 (Ven

In [18]:
# Generazione di 40 dataset diversi
datasets = []

# Configurazione base per i dataset
def configura_dataset(titoli_dataset, colonne_dataset, indice):

    nome_file_dataset = titoli_dataset[indice].replace(" ", "_") + ".csv"

    return {
        "titolo": titoli_dataset[indice],
        "nome_file": nome_file_dataset,
        "colonne": colonne_dataset[indice]
    }

# Creazione dei 40 dataset
for i in range(0, 27):
    datasets.append(configura_dataset(titoli_dataset, colonne_dataset, i))

# Generazione e salvataggio di ogni dataset
i=0
dfs = []
for dataset in datasets:
    i=i+1
    df = genera_dataset(dataset, 1000)  # Genera 1000 righe
    df.to_csv(dataset['nome_file'], index=False)
    print(f"Dataset {i} {dataset['titolo']} salvato in {dataset['nome_file']}")
    dfs.append(df)


Dataset 1 Meteo Storico salvato in Meteo_Storico.csv
Dataset 2 Vendite Giornaliere salvato in Vendite_Giornaliere.csv
Dataset 3 Traffico Veicolare salvato in Traffico_Veicolare.csv
Dataset 4 Consumi Elettrici salvato in Consumi_Elettrici.csv
Dataset 5 Qualità dell'Aria salvato in Qualità_dell'Aria.csv
Dataset 6 Produzione Agricola salvato in Produzione_Agricola.csv
Dataset 7 Prestazioni Studenti salvato in Prestazioni_Studenti.csv
Dataset 8 Eventi Climatici salvato in Eventi_Climatici.csv
Dataset 9 Vendite Ecommerce salvato in Vendite_Ecommerce.csv
Dataset 10 Salute Pubblica salvato in Salute_Pubblica.csv
Dataset 11 Dati Sanitario Regionale salvato in Dati_Sanitario_Regionale.csv
Dataset 12 Statistiche Turistiche salvato in Statistiche_Turistiche.csv
Dataset 13 Performance Aziendali salvato in Performance_Aziendali.csv
Dataset 14 Andamento Mercati Finanziari salvato in Andamento_Mercati_Finanziari.csv
Dataset 15 Comunicazioni Social Media salvato in Comunicazioni_Social_Media.csv
Datas

In [20]:
import pandas as pd

# ... (codice esistente per generare i DataFrame in dfs) ...

for i, df in enumerate(dfs):
    print(f"DataFrame {i + 1} - {titoli_dataset[i]}:")

    # Seleziona le colonne binarie (colonne con 'tipo': "binario" nella configurazione)
    binary_cols = [col['nome'] for col_list in colonne_dataset for col in col_list if col['tipo'] == 'binario']

    # Stampa la frequenza per ogni colonna binaria
    for col in binary_cols:
        if col in df.columns:  # Verifica se la colonna esiste nel DataFrame
            print(f"  Colonna '{col}':")
            freq = df[col].value_counts(normalize=True, dropna=False)

            # Formattazione in percentuale, gestendo potenziali KeyError
            true_freq = freq.get(True, 0) * 100
            false_freq = freq.get(False, 0) * 100
            na_freq = freq.get(pd.NA, 0) * 100

            print(f"    True: {true_freq:.2f}%, False: {false_freq:.2f}%, NA: {na_freq:.2f}%")

    print("\n")

DataFrame 1 - Meteo Storico:
  Colonna 'Allerta Tempesta':
    True: 47.10%, False: 47.90%, NA: 5.00%


DataFrame 2 - Vendite Giornaliere:
  Colonna 'Vendite Raggiunte Obiettivo':
    True: 47.80%, False: 47.20%, NA: 5.00%


DataFrame 3 - Traffico Veicolare:
  Colonna 'Allerta Traffico':
    True: 46.10%, False: 48.90%, NA: 5.00%


DataFrame 4 - Consumi Elettrici:
  Colonna 'Anomalia Consumo':
    True: 47.90%, False: 47.10%, NA: 5.00%


DataFrame 5 - Qualità dell'Aria:
  Colonna 'Allerta Smog':
    True: 48.70%, False: 46.30%, NA: 5.00%


DataFrame 6 - Produzione Agricola:
  Colonna 'Successo Produzione Stagionale':
    True: 48.80%, False: 46.20%, NA: 5.00%


DataFrame 7 - Prestazioni Studenti:
  Colonna 'Successo Accademico':
    True: 47.70%, False: 47.30%, NA: 5.00%


DataFrame 8 - Eventi Climatici:
  Colonna 'Evento Estremo':
    True: 47.40%, False: 47.60%, NA: 5.00%


DataFrame 9 - Vendite Ecommerce:
  Colonna 'Transazione Rimborsata':
    True: 47.50%, False: 47.50%, NA: 5.00%

In [22]:
!mkdir datasets

In [23]:
!mv *.csv datasets/

In [24]:
from google.colab import drive
import shutil

# Monta il tuo Google Drive
drive.mount('/content/drive')

# Definisci il percorso di origine e destinazione
source_path = '/content/datasets'  # Assumi che la cartella si chiami "datasets"
destination_path = '/content/drive/My Drive/datasets'  # Cambia "datasets" con il nome desiderato

# Sposta la cartella
shutil.move(source_path, destination_path)

print(f"Cartella 'datasets' spostata in '{destination_path}'")

Mounted at /content/drive
Cartella 'datasets' spostata in '/content/drive/My Drive/datasets'
